# Ensemble - Boosting Model
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다. 즉 **약한 학습기들은 앞 학습기가 만든 오류를 줄이는 방향으로 학습한다**

# GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassifier, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 안 좋은 단점이 있다.

### 주요 파라미터
- **Decision Tree 의 가지치기 관련 매개변수**
    - 각각의 decision tree가 복잡한 모델이 되지 않도록 한다. 

- **learning rate**
    - 이전 decision tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1

- **n_estimators**
    - decision tree의 개수 지정. 많을 수록 복잡한 모델이 된다.

- **n_iter_no_change, validation_fraction**
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아 지지 않으면 훈련을 조기 종료한다.

- **보통 max_depth를 낮춰 개별 decision tree의 복잡도를 낮춘다. 보통 5가 넘지 않게 설정한다. 그리고 n_estimators를 가용시간, 메모리 한도에 맞춰 크게 설정하고 적절한 learning_rate을 찾는다.**

### 예제

In [3]:
from metrics import print_classification_metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X, y = data['data'], data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

##### GradientBoostingClassifier 모델 생성, 학습, 평가

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=0)

gb.fit(X_train, y_train)

pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_classification_metrics(y_train, pred_train, title='Train')
print_classification_metrics(y_test, pred_test, title='Test')

Train
--------------------------------------------------
정확도(Accuracy): 1.0
재현율/민감도(Recall): 1.0
정밀도(Precision): 1.0
F1 점수(F1 Score): 1.0
Test
--------------------------------------------------
정확도(Accuracy): 0.958041958041958
재현율/민감도(Recall): 0.9555555555555556
정밀도(Precision): 0.9772727272727273
F1 점수(F1 Score): 0.9662921348314608


##### 하이퍼파라미터 변경에 따른 성능 비교

In [5]:
gb = GradientBoostingClassifier(random_state=0, max_depth=1) #learning_rate: 0.1 (기본), n_estimators=100(기본)

gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_classification_metrics(y_train, pred_train, title='Train')
print_classification_metrics(y_test, pred_test, title='Test')

Train
--------------------------------------------------
정확도(Accuracy): 0.9953051643192489
재현율/민감도(Recall): 1.0
정밀도(Precision): 0.9925650557620818
F1 점수(F1 Score): 0.9962686567164178
Test
--------------------------------------------------
정확도(Accuracy): 0.965034965034965
재현율/민감도(Recall): 0.9777777777777777
정밀도(Precision): 0.967032967032967
F1 점수(F1 Score): 0.9723756906077348


In [6]:
gb = GradientBoostingClassifier(random_state=0, max_depth=1, learning_rate=0.1, n_estimators=1000) #learning_rate 기본값: 0.1 => 0.001 (과소적합)

gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_classification_metrics(y_train, pred_train, title='Train')
print_classification_metrics(y_test, pred_test, title='Test')

Train
--------------------------------------------------
정확도(Accuracy): 1.0
재현율/민감도(Recall): 1.0
정밀도(Precision): 1.0
F1 점수(F1 Score): 1.0
Test
--------------------------------------------------
정확도(Accuracy): 0.958041958041958
재현율/민감도(Recall): 0.9555555555555556
정밀도(Precision): 0.9772727272727273
F1 점수(F1 Score): 0.9662921348314608


In [7]:
gb = GradientBoostingClassifier(random_state=0, max_depth=1, learning_rate=0.01, n_estimators=1000) #learning_rate 기본값: 0.1 => 0.01 (과소적합)

gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_classification_metrics(y_train, pred_train, title='Train')
print_classification_metrics(y_test, pred_test, title='Test')

Train
--------------------------------------------------
정확도(Accuracy): 0.9953051643192489
재현율/민감도(Recall): 1.0
정밀도(Precision): 0.9925650557620818
F1 점수(F1 Score): 0.9962686567164178
Test
--------------------------------------------------
정확도(Accuracy): 0.958041958041958
재현율/민감도(Recall): 0.9666666666666667
정밀도(Precision): 0.9666666666666667
F1 점수(F1 Score): 0.9666666666666667


##### Feature 중요도를 조회

In [8]:
gb.feature_importances_

array([0.        , 0.00151889, 0.        , 0.        , 0.        ,
       0.        , 0.00446665, 0.16350925, 0.        , 0.        ,
       0.00044232, 0.        , 0.        , 0.00377378, 0.00122931,
       0.        , 0.00041146, 0.        , 0.        , 0.        ,
       0.09167029, 0.01931462, 0.30713717, 0.10577961, 0.00404029,
       0.        , 0.00739352, 0.28808604, 0.0012268 , 0.        ])

In [9]:
import pandas as pd
fi = pd.Series(gb.feature_importances_, index=data['feature_names'])
fi.sort_values(ascending=False)

worst perimeter            0.307137
worst concave points       0.288086
mean concave points        0.163509
worst area                 0.105780
worst radius               0.091670
worst texture              0.019315
worst concavity            0.007394
mean concavity             0.004467
worst smoothness           0.004040
area error                 0.003774
mean texture               0.001519
smoothness error           0.001229
worst symmetry             0.001227
radius error               0.000442
concavity error            0.000411
worst compactness          0.000000
fractal dimension error    0.000000
mean radius                0.000000
compactness error          0.000000
symmetry error             0.000000
concave points error       0.000000
perimeter error            0.000000
texture error              0.000000
mean fractal dimension     0.000000
mean symmetry              0.000000
mean compactness           0.000000
mean smoothness            0.000000
mean area                  0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

##### RandomizedSearchCV 생성

In [10]:
from sklearn.model_selection import RandomizedSearchCV

param = {
    'n_estimators':range(500, 1001, 100),
    'learning_rate':[0.001, 0.05, 0.01, 0.1, 0.5], 
    'max_depth':[1,2,3],
    'subsample':[0.5, 0.7, 0.9, 1]
}

rs = RandomizedSearchCV(GradientBoostingClassifier(random_state=0),
                        param,
                        cv=4, 
                        n_iter=60, 
                        scoring='accuracy', 
                        n_jobs=-1)

##### 학습, 평가

In [11]:
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=4, estimator=GradientBoostingClassifier(random_state=0),
                   n_iter=60, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.05, 0.01,
                                                          0.1, 0.5],
                                        'max_depth': [1, 2, 3],
                                        'n_estimators': range(500, 1001, 100),
                                        'subsample': [0.5, 0.7, 0.9, 1]},
                   scoring='accuracy')

##### best estimator 조회 및 Test set 최종평가

In [12]:
rs.best_params_

{'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 2, 'learning_rate': 0.1}

In [13]:
best_model = rs.best_estimator_
pred_test = best_model.predict(X_test)

In [14]:
print_classification_metrics(y_test, pred_test, "best_model")

best_model
--------------------------------------------------
정확도(Accuracy): 0.958041958041958
재현율/민감도(Recall): 0.9666666666666667
정밀도(Precision): 0.9666666666666667
F1 점수(F1 Score): 0.9666666666666667


# XGBoost(Extra Gradient Boost)
- https://readthedocs.org/projects/xgboost/
- Gradient Boost 알고리즘을 기반으로 개선해서 분산환경에서도 실행할 수 있도록 구현 나온 모델.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제들을 제공하여 성능을 높임.
- 회귀와 분류 모두 지원한다.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것을 알려저 유명해짐.
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치
``
conda install -y -c anaconda py-xgboost
``

## Scikit-learn 래퍼(Wrapper) XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- Decision Tree관련 하이퍼파라미터들

### 예제

In [16]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=2, random_state=0)

xgb.fit(X_train, y_train)

pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)

print_classification_metrics(y_train, pred_train, "XGB Train")
print_classification_metrics(y_test, pred_test, 'XGB Test')

E:\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:15:57] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB Train
--------------------------------------------------
정확도(Accuracy): 0.9953051643192489
재현율/민감도(Recall): 0.9962546816479401
정밀도(Precision): 0.9962546816479401
F1 점수(F1 Score): 0.9962546816479401
XGB Test
--------------------------------------------------
정확도(Accuracy): 0.951048951048951
재현율/민감도(Recall): 0.9555555555555556
정밀도(Precision): 0.9662921348314607
F1 점수(F1 Score): 0.9608938547486034


# Ensemble - Voting 방식
- 서로 다른 종류의 알고리즘들을 결합하여 다수결 방식으로 최종 결과를 출력한다.

## Voting의 유형 (분류)
1. **hard voting**
    - 다수의 추정기가 결정한 예측값들 중 많은 것을 선택하는 방식
2. **soft voting**
    - 다수의 추정기에서 각 레이블별 예측한 확률들의 평균을 내서 높은 레이블값을 결과값으로 선택하는 방식

- 일반적으로 soft voting이 성능이 더 좋다.    
- Voting은 성향이 다르면서 비슷한 성능을 가진 모델들을 묶었을때 가장 좋은 성능을 낸다.
    - 성능이 비슷하면서 다른 예측을 하는 모델들을 묶었을 때 가장 좋은 성능을 낸다.

## VotingClassifier 클래스 이용
- 매개변수
    - estimators : 앙상블할 모델들 설정.  ("추정기이름", 추정기) 의 튜플을 리스트로 묶어서 전달
    - voting: voting 방식. hard(기본값), soft  지정

##### import

In [17]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

##### Data전처리
- SVM, KNN, LogisticRegression은 Feature Scaling 전처리 데이터를 사용
- Random Forest, XGBoost는 Decision Tree기반이므로 Feature Scaling이 필요없다.

In [18]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### 모델들 생성, 학습, 평가

In [19]:
# 평가함수
def print_metrics(y, pred, title=None):
    acc = accuracy_score(y, pred)
    if title:
        print(title)
    print("정확도: ", acc)

In [20]:
# 모델들 객체를 생성
svc = SVC(random_state=0, probability=True) 
knn = KNeighborsClassifier(n_neighbors=5)
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=0)
lr = LogisticRegression(max_iter=1000, random_state=0)
xgb = XGBClassifier(max_depth=2, random_state=0)

# 각 모델들을 학습
svc.fit(X_train_scaled, y_train) # Feature Scaling 된 X_train
knn.fit(X_train_scaled, y_train) # Feature Scaling 된 X_train
rf.fit(X_train, y_train)
lr.fit(X_train_scaled, y_train) # Feature Scaling 된 X_train
xgb.fit(X_train, y_train)

# 추론
pred_train_svc = svc.predict(X_train_scaled)
pred_train_knn = knn.predict(X_train_scaled)
pred_train_rf = rf.predict(X_train)
pred_train_lr = lr.predict(X_train_scaled)
pred_train_xgb = xgb.predict(X_train)

pred_test_svc = svc.predict(X_test_scaled)
pred_test_knn = knn.predict(X_test_scaled)
pred_test_rf = rf.predict(X_test)
pred_test_lr = lr.predict(X_test_scaled)
pred_test_xgb = xgb.predict(X_test)

E:\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:23:49] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [21]:
# trainset 평가
print_metrics(y_train, pred_train_svc, "Train SVC")
print_metrics(y_train, pred_train_knn, "Train KNN")
print_metrics(y_train, pred_train_rf, "Train RF")
print_metrics(y_train, pred_train_lr, "Train LogisticRegression")
print_metrics(y_train, pred_train_xgb, 'Train XGB')

Train SVC
정확도:  0.9929577464788732
Train KNN
정확도:  0.9788732394366197
Train RF
정확도:  0.9976525821596244
Train LogisticRegression
정확도:  0.9906103286384976
Train XGB
정확도:  1.0


In [22]:
# testset 평가
print_metrics(y_test, pred_test_svc, 'Test SVC')
print_metrics(y_test, pred_test_knn, 'Test KNN')
print_metrics(y_test, pred_test_rf, 'Test RF')
print_metrics(y_test, pred_test_lr, 'Test LogisticRegression')
print_metrics(y_test, pred_test_xgb, 'Test XGBoost')

# 1. 성능이 비슷?
# 2. 결과에 차이가 있나? (다른 예측?) ==> ?(상관관계 분석)

Test SVC
정확도:  0.958041958041958
Test KNN
정확도:  0.951048951048951
Test RF
정확도:  0.9440559440559441
Test LogisticRegression
정확도:  0.958041958041958
Test XGBoost
정확도:  0.9440559440559441


### 앙상블 상관관계
- 상관관계가 높은 모델을 앙상블에 포함시키는 것은 바람직 하지 않다.
- 모델간의 상관관계가 높다는 것은 두 모델이 동일한 예측을 한다는 것이다. 같은 예측을 하는 모델은 의미가 없다. 
- Voting방식(다수결 투표방식)의 앙상블은 각각 좋은 성능을 내지만 다른 예측을 하는 다양한 모델을 모아서 하는 것이 좋다.  대부분의 모델들이 동일한 예측을 만든다면 새로운 모델을 추가해 얻는 이득이 적다.

#### 모델간의 상관관계 확인
- 각 모델의 예측 결과를 이용해 상관계수를 구한다.

In [23]:
import numpy as np

a = np.array([1,2,3])
b = np.array([10,20,30])
c = np.array([100,200,300])

# 1차원 배열들을 2차원 배열이 되도록 합치기. 1축 기준으로 합칠때  c_[배열, 배열, 배열]
print(np.c_[a, b, c])
# 1차원 배열들을 0축을 기준으로 합치기. r_[배열, 배열, 배열]

[[  1  10 100]
 [  2  20 200]
 [  3  30 300]]


In [24]:
import numpy as np
import pandas as pd
# Train set에대한 예측 결과를 기준
df = pd.DataFrame(np.c_[pred_train_svc,pred_train_knn,pred_train_rf ,pred_train_lr,pred_train_xgb], 
                  columns=["SVM", "KNN", 'RandomForest', 'LogisticRegression', 'XGBoost'])
df.head()

,SVM,KNN,RandomForest,LogisticRegression,XGBoost
0,1,1,1,1,1
1,0,0,0,0,0
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1


In [25]:
# 각 feature가의 상관계수 계산 - df.corr()
df.corr()

,SVM,KNN,RandomForest,LogisticRegression,XGBoost
SVM,1.000000,0.969819,0.989940,0.994976,0.984941
KNN,0.969819,1.000000,0.959758,0.974829,0.954799
RandomForest,0.989940,0.959758,1.000000,0.984902,0.994989
LogisticRegression,0.994976,0.974829,0.984902,1.000000,0.979929
XGBoost,0.984941,0.954799,0.994989,0.979929,1.000000


In [26]:
df_test = pd.DataFrame(np.c_[pred_test_svc,pred_test_knn,pred_test_rf ,pred_test_lr,pred_test_xgb], 
                  columns=["SVM", "KNN", 'RandomForest', 'LogisticRegression', 'XGBoost'])
df_test.head()

,SVM,KNN,RandomForest,LogisticRegression,XGBoost
0,1,1,1,1,1
1,0,0,0,0,0
2,0,0,0,0,0
3,1,1,1,1,1
4,0,0,0,0,0


In [27]:
df_test.corr()

,SVM,KNN,RandomForest,LogisticRegression,XGBoost
SVM,1.000000,0.895618,0.880084,0.970021,0.940042
KNN,0.895618,1.000000,0.926280,0.895618,0.895618
RandomForest,0.880084,0.926280,1.000000,0.880084,0.910063
LogisticRegression,0.970021,0.895618,0.880084,1.000000,0.940042
XGBoost,0.940042,0.895618,0.910063,0.940042,1.000000


##### VotingClassifier로 앙상블

In [28]:
# 앙상블할 모델들 리스트 
estimators = [
    ('knn', knn),
    ('xgb', xgb), 
    ('random forest', rf), 
    ('svm', svc)
]

###### hard voting

In [29]:
voting = VotingClassifier(estimators) #hard voting

voting.fit(X_train_scaled, y_train)  #각 모델을 학습시킨다.

pred_train = voting.predict(X_train_scaled)
pred_test = voting.predict(X_test_scaled)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

E:\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:30:02] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
정확도:  0.9953051643192489
정확도:  0.9440559440559441


###### soft voting

In [30]:
voting = VotingClassifier(estimators, voting='soft') #soft voting (확률 평균으로 결과를 예측)
voting.fit(X_train_scaled, y_train)

pred_train = voting.predict(X_train_scaled)
pred_test = voting.predict(X_test_scaled)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

E:\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:30:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
정확도:  0.9953051643192489
정확도:  0.958041958041958


##### Pipeline을 이용해 데이터전처리와 모델 묶어서 처리
- KNN, SVM: Feature Scaling이 필요 => Pipeline
- XGBoost, Random Forest:  Feature Scaling 불필요

In [32]:
from sklearn.pipeline import Pipeline, make_pipeline
# 파이프라인 생성
order_knn = [
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]
order_svm = [
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=0, probability=True))
]

knn_pl = Pipeline(order_knn)  # make_pipeline(SS(), KNN())
svm_pl = Pipeline(order_svm)

# XGBoost
xgb = XGBClassifier(max_depth=2, random_state=0)
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=0)

estimators = [
    ('knn', knn_pl), ('svm', svm_pl), ('xgb', xgb), ('random forest', rf)
]
voting = VotingClassifier(estimators, voting='soft')

voting.fit(X_train, y_train)

print_metrics(y_train, voting.predict(X_train))
print_metrics(y_test, voting.predict(X_test))

E:\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:34:53] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
정확도:  0.9953051643192489
정확도:  0.958041958041958
